In [79]:
import yaml, subprocess, os, json, sys
from datetime import datetime
import pandas as pd
import numpy as np
from dotenv import dotenv_values
import openstack
env = dotenv_values(".env")

def create_connection(auth_url, region, project_name, username, password,
                      user_domain, project_domain):
    return openstack.connect(
        auth_url=auth_url,
        project_name=project_name,
        username=username,
        password=password,
        region_name=region,
        user_domain_name=user_domain,
        project_domain_name=project_domain,
        app_name='bpet',
        app_version='1.0',
    )
    
def collect_info(WATCHDOG_ADDRESS, key):
    conn = create_connection(auth_url=env['OS_AUTH_URL'], region=env['OS_REGION_NAME'],
        project_name=env['OS_PROJECT_NAME'], username=env['OS_USERNAME'],
        password=env['OS_PASSWORD'], user_domain=env['OS_USER_DOMAIN_NAME'],
        project_domain=env['OS_PROJECT_ID'])
    
    subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", "get_nodeinfo.py",
    "ubuntu@{}:/home/ubuntu/".format(WATCHDOG_ADDRESS)])

    COMMAND = 'python3 get_nodeinfo.py'
    subprocess.Popen(["ssh", "-i", key, 
                        "-o", "StrictHostKeyChecking=no", "ubuntu@%s" % WATCHDOG_ADDRESS, COMMAND],
                        shell=False,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE)
    subprocess.run(['sleep', '3'])
    subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
        "ubuntu@{}:/home/ubuntu/nodeinfo.json".format(WATCHDOG_ADDRESS), os.getcwd()])

    subprocess.run(['sleep', '2'])

    with open('nodeinfo.json', 'r') as f:
        data = json.load(f)
        rows = data['nodeinfo']

    for row in rows:
        instance_id = conn.compute.find_server(row[0]).id
        host_id = conn.compute.get_server(instance_id).host_id
        row.append(instance_id)
        row.append(host_id)
    df = pd.DataFrame(np.array(rows),
        columns=['NodeName', 'IP', 'NodeAddress', 'IsValidator', 'InstanceID', 'HostID'])
    # sort dataframe
    df['Index'] = [int(name.split('-')[1]) for name in df.NodeName]
    df = df.set_index(keys=df.Index).drop(labels='Index', axis=1).sort_index()
    
    with open('nodeinfo.json', 'w') as f:
        json.dump({'nodeinfo': rows}, f, indent=4)
    return df

def run(WATCHDOG_ADDRESS, key, SEND_RATES):
    df = collect_info(WATCHDOG_ADDRESS, key)

    DEFAULT_IP = df.IP.values[0]
    connection_url = "ws://" + DEFAULT_IP + ":8546"
    # if len(sys.argv) > 1:
    #     connection_url = "ws://" + sys.argv[1] + ":8546"
    networkconfig = 'networks/4node-ibft2/networkconfig.json'
    with open(networkconfig, 'r') as f:
        data = json.load(f)
        data['ethereum']['url'] = connection_url
    with open(networkconfig, 'w') as f:
        json.dump(data, f, indent=4)

    timestampStr = datetime.now().strftime("%Y%m%d-%H%M%S")
    directory = 'reports/' + timestampStr
    path = os.path.join(os.getcwd(), directory)
    os.mkdir(path)

    replicas = 5 # test replicas for each send rate
    rounds = 3 # test rounds: open, query and transfer
    for tps in SEND_RATES:
        with open('benchmarks/scenario/simple/config.yaml','r') as f:
            y=yaml.safe_load(f)
            for i in range(rounds):
                y['test']['rounds'][i]['rateControl']['opts']['tps'] = tps
        with open('benchmarks/scenario/simple/config.yaml','w') as f:
            yaml.dump(y,f,default_flow_style=False, sort_keys=False)

        for i in range(replicas):
            subprocess.run(['docker-compose', 'up'])
            subprocess.run(['cp', 'report.html', '{}/report-{}-{}.html'.format(directory, tps, i+1)])
            subprocess.run(['sleep', '10'])

    print(df)

    subprocess.run(['sleep', '10'])

    for _, row in df.iterrows():
        COMMAND = 'docker logs $(docker ps -q) > {}.log'.format(row['NodeName'])
        subprocess.Popen(["ssh", "-i", key, 
                            "-o", "StrictHostKeyChecking=no", "ubuntu@%s" % row['IP'], COMMAND],
                            shell=False,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

    subprocess.run(['sleep', '10'])

    collect_log(df, key)

def collect_log(df, key):
    timestampStr = datetime.now().strftime("%Y%m%d-%H%M%S")
    directory = '../data/logs-' + timestampStr
    log_path = os.path.join(os.getcwd(), directory)
    os.mkdir(log_path)
    # collect besu logs
    for _, row in df.iterrows():
        subprocess.run(['scp', '-i', key, "-o", "StrictHostKeyChecking=no", 
        "ubuntu@{}:/home/ubuntu/{}.log".format(row['IP'], row.NodeName), log_path])
    # collect caliper logs
    subprocess.run(['mv', 'caliper.log', log_path])
    # collect node info json data
    subprocess.run(['mv', 'nodeinfo.json', log_path])
    

# watchdogAddress = "192.168.226.176"
watchdogAddress = "10.2.1.9"
# keyFile = "../data/rrg-bpet"
keyFile = "../data/bpet.pem"
current_directory = os.getcwd()
sshKey = os.path.join(current_directory, keyFile)
sendRates = [50, 100, 150, 200, 250]
df = collect_info(watchdogAddress, sshKey)
df

,NodeName,IP,NodeAddress,IsValidator,InstanceID,HostID
Index,,,,,,
1,besu-1,10.2.8.94,0x1631b2b52e3529b56e002d0e39410899a6530678,True,8fe7c34e-4b08-4f1c-85c7-3da72a1bfc35,249eb13a931a5ffac2ef3b3c9d5188a2e87a0772cf068a...
2,besu-2,10.2.12.162,0xbcc0a459af33d3255dd79a3748a609731dbf4a53,True,d6410d6d-99b7-4059-a3bb-3643e6385fb1,24c113f9c84e4a4f346e00fefefb5510f40736a41ce903...
3,besu-3,10.2.7.52,0xbe1a5dc2d5d4f3407494b313ff2065d713027e2a,True,4229f5b7-0028-4a30-9ad8-32ad8a8e9085,b63081ee267952efbac4c2c0f0127e067bd4a385bdc810...
4,besu-4,10.2.11.39,0x15156305bf9670c9c33651e49d3b822da2971fc9,True,49cd3bab-11e4-4bb9-a750-07c933d898ce,17497a9a09a63cd570b1d839818b807207919152f9438e...
5,besu-5,10.2.12.112,0x3134cd35d91f2c1f7d61a5ca16c52c0c9c42143d,True,610930c7-e97c-448b-b5fe-75181285d1d4,5bc3c72bf1b42515cb454e42d4bc94b7b7acd3f12519d0...
6,besu-6,10.2.11.83,0x33534f5e221a3298005e300326bedb6466b3549e,True,668e5471-a37a-46c6-a932-b1bc04e1b398,12e678d36d6dc54feee7fb00fe48678bab6fd26a76a20d...


In [3]:
from dotenv import dotenv_values
env = dotenv_values(".env")
import openstack
def create_connection(auth_url, region, project_name, username, password,
                      user_domain, project_domain):
    return openstack.connect(
        auth_url=auth_url,
        project_name=project_name,
        username=username,
        password=password,
        region_name=region,
        user_domain_name=user_domain,
        project_domain_name=project_domain,
        app_name='bpet',
        app_version='1.0',
    )
conn = create_connection(auth_url=env['OS_AUTH_URL'], region=env['OS_REGION_NAME'],
        project_name=env['OS_PROJECT_NAME'], username=env['OS_USERNAME'],
        password=env['OS_PASSWORD'], user_domain=env['OS_USER_DOMAIN_NAME'],
        project_domain=env['OS_PROJECT_DOMAIN_NAME'])

In [7]:
for server in conn.compute.servers():
    print(server.name, server.host_id)

besu-8 b63081ee267952efbac4c2c0f0127e067bd4a385bdc81005cc689853
besu-3 17497a9a09a63cd570b1d839818b807207919152f9438e6a8aec6de0
besu-5 24c113f9c84e4a4f346e00fefefb5510f40736a41ce903fb5b165d9a
besu-4 5bc3c72bf1b42515cb454e42d4bc94b7b7acd3f12519d0cd0ac8ea70
besu-7 12e678d36d6dc54feee7fb00fe48678bab6fd26a76a20db6cb95d7bb
besu-6 17497a9a09a63cd570b1d839818b807207919152f9438e6a8aec6de0
besu-2 249eb13a931a5ffac2ef3b3c9d5188a2e87a0772cf068a754b7022b4
besu-1 5bc3c72bf1b42515cb454e42d4bc94b7b7acd3f12519d0cd0ac8ea70
watchdog 5bc3c72bf1b42515cb454e42d4bc94b7b7acd3f12519d0cd0ac8ea70
besu-base 249eb13a931a5ffac2ef3b3c9d5188a2e87a0772cf068a754b7022b4


In [10]:
import json
with open('nodeinfo.json') as f:
    data = json.load(f)
    rows = data['nodeinfo']
for row in rows:
    instance_id = conn.compute.find_server(row[0]).id
    host_id = conn.compute.get_server(instance_id).host_id
    row.append(instance_id)
    row.append(host_id)
with open('nodeinfo.json', 'w') as f:
    json.dump({"nodeinfo": rows}, f, indent=4)